In [213]:
import pandas as pd
import numpy as np
import sklearn.utils
import imblearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [214]:
df = pd.read_csv('nfa.csv')
df.head(5)

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,qscore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [215]:
df['qscore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: qscore, dtype: int64

In [216]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
qscore                1
dtype: int64

In [217]:
df = df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
qscore            0
dtype: int64

In [218]:
df['qscore'].value_counts()

3A    51473
2A      224
1A       16
Name: qscore, dtype: int64

In [219]:
df['qscore'] = df['qscore'].replace(['1A'], '2A')
df['qscore'].value_counts()

3A    51473
2A      240
Name: qscore, dtype: int64

In [220]:
df_2A = df[df.qscore == '2A']
df_3A = df[df.qscore == '3A'].sample(350)

data = df_2A.append(df_3A)
data['qscore'].value_counts()

3A    350
2A    240
Name: qscore, dtype: int64

In [221]:
data = sklearn.utils.shuffle(data)

In [222]:
data = data.reset_index(drop=True)

In [223]:
data.shape

(590, 12)

In [224]:
#more preprocessing

data = data.drop(columns = ['country_code', 'year', 'country'])

In [225]:
X = data.drop(columns = 'qscore')
y = data['qscore']

In [226]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [227]:
y_train.value_counts()

3A    233
2A    180
Name: qscore, dtype: int64

In [228]:
encoder =  LabelEncoder()

In [229]:
X_train.record = encoder.fit_transform(X_train.record)
X_test.record = encoder.transform(X_test.record)

In [230]:
smote = SMOTE(random_state = 1)
X_train_balanced, y_balanced = smote.fit_resample(X_train, y_train)

In [232]:
X_test = X_test.reset_index(drop= True )

normalised_test_df = scaler.transform(X_test.drop(columns=[ 'record' ]))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=X_test.drop(columns=[ 'record' ]).columns)
normalised_test_df[ 'record' ] = X_test[ 'record' ]

In [233]:
#Logistic Regression

log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

#returns
LogisticRegression(C= 1.0 , class_weight= None , dual= False , fit_intercept= True , intercept_scaling= 1 , l1_ratio= None , max_iter= 100 , multi_class= 'auto' , n_jobs= None , penalty= 'l2' ,
random_state= None , solver= 'lbfgs' , tol= 0.0001 , verbose= 0 , warm_start= False )

LogisticRegression()

In [234]:
#checking cross-validation

scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv= 5 , scoring= 'f1_macro' )
scores

array([0.55777395, 0.54707792, 0.57751893, 0.37627197, 0.61925371])

In [235]:
#K-FOLD

# kf = KFold(n_splits= 5 )
# kf.split(normalised_train_df)

# f1_scores = []

# #run for every split
# for train_index, test_index in kf.split(normalised_train_df):
#     X_train, X_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    
#     y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    
#     model = LogisticRegression().fit(X_train, y_train)

# #save result to list
#     f1_scores.append(f1_scores (y_true = y_test, y_pred=model.predict(X_test), pos_label= '2A')*100)

In [236]:
# skf = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1 )

# f1_scores = []

# #run for every split
# for train_index, test_index in skf.split(normalised_train_df, y_balanced):
#     X_train, X_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    
#     y_train, y_test = y_balanced[train_index], y_balanced[test_index]

#     model = LogisticRegression().fit(X_train, y_train)
# #save result to list
#     f1_scores.append(f1_scores(y_true=y_test, y_pred=model.predict(X_test), pos_label= '2A' ))

In [237]:
loo = LeaveOneOut()

scores = cross_val_score(LogisticRegression(), normalised_train_df, 
                        y_balanced, cv = loo, scoring = 'f1_macro')
average_score = scores.mean() * 100


In [238]:
print(average_score)

48.06866952789699


In [239]:
new_predictions = log_reg.predict(normalised_test_df)

cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=[ '2A' , '3A' ])
cnf_mat


array([[29, 31],
       [51, 66]], dtype=int64)

In [240]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 ))

Accuracy: 54


In [241]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 36


In [242]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 48


In [243]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 41


In [244]:
dec_tree = DecisionTreeClassifier()

dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()